In [7]:
import numpy as np
import pandas as pd
import time
from scipy.optimize import minimize

In [2]:
## Read in csv file from IESO website, ***fill blank entries with 0*** (temp)
data_6months = pd.read_excel("SampleWeatherData.xlsx", index_col=0)
data_6months ['Month 1&2'] = data_6months ['Month 1&2'].apply(lambda x: (2.02887*x + 39.633)/100)
data_6months ['Month 3&4'] = data_6months ['Month 3&4'].apply(lambda x: (0.453122*x + 19.8276)/100)
data_6months ['Month 5&6'] = data_6months ['Month 5&6'].apply(lambda x: (1.13665*x - 11.0085)/100)
data_6months ['Month 7&8'] = data_6months ['Month 7&8'].apply(lambda x: (1.90245*x - 23.2826)/100)
data_6months ['Month 9&10'] = data_6months ['Month 9&10'].apply(lambda x: (1.39145*x - 8.97971)/100)
data_6months ['Month 11&12'] = data_6months ['Month 11&12'].apply(lambda x: (1.72767*x + 21.3536)/100)


In [3]:
load_data = pd.read_excel('load_data.xlsx', index_col=0)


In [8]:
## x[0:24] = PCEA
## x[24:48] = ESB
## x[48:72] = PCDDR
start_time = time.time()

def constraint1(x):
    for i in range(24):
        if (i == 0):
            x[24] = 0
        else:
            x[24+i] = x[24+i-1] + x[i]
    return x[0:24] + x[24:48] - x[48:72]
    
def constraint2(x):
    for i in range(24):
        if (i == 0):
            x[24] = 0
        else:
            x[24+i] = x[24+i-1] + x[i]
    return 10000 - (x[0:24]+ x[24:48]- x[48:72])
   
    

power = ((-5000, 5000),) * 24
storage = ((0, 10000),) * 24
DDR = ((0,5000),) * 24

KRDDR = 856.436/8760

##Objective
def MEC(x):           #      (    PDLL             +             PCEA     )          x        HOEP
    return -(KRDDR*sum(30*sum(np.array([x[48:72]]))) - \
            sum(30*sum((load_data.iloc[:,month].to_numpy() + np.array([x[0:24]])) * data_6months.iloc[:,month].to_numpy())))


x0 = np.array([np.ones(24), np.ones(24), np.ones(24)])
bounds = (power + storage + DDR)
cons1 = {'type': 'ineq', 'fun': constraint1}
cons2 = {'type': 'ineq', 'fun': constraint2}

cons = ([cons1, cons2])

pcea_solutions = []
esb_solutions = []
pcddr_solutions = []

mec = 0
for month in range(6):
    sol = minimize(MEC, x0, method='SLSQP',
                   bounds=bounds,
                   constraints=cons,
                   options= {'maxiter':150,'disp':True})
    pcea_solutions.append(sol.x[0:24])
    esb_solutions.append(sol.x[24:48])
    pcddr_solutions.append(sol.x[48:72])
    mec = mec + sol.fun
    
print(mec)
print("--- %s seconds ---" % (time.time() - start_time))


Iteration limit exceeded    (Exit mode 9)
            Current function value: 19543.528206449933
            Iterations: 151
            Function evaluations: 10878
            Gradient evaluations: 147
Iteration limit exceeded    (Exit mode 9)
            Current function value: -226808.2129823804
            Iterations: 151
            Function evaluations: 10878
            Gradient evaluations: 147
Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: -293158.16687326087
            Iterations: 109
            Function evaluations: 7770
            Gradient evaluations: 105
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -154531.26232208885
            Iterations: 129
            Function evaluations: 9250
            Gradient evaluations: 125
Iteration limit exceeded    (Exit mode 9)
            Current function value: -256711.76090023597
            Iterations: 151
            Function evalua

In [ ]:
df_sol.columns = load_data.columns.values.tolist() + load_data.columns.values.tolist() + load_data.columns.values.tolist()
df_sol

In [ ]:
df_sol = pd.concat([pd.DataFrame(data=pcea_solutions).transpose(), pd.DataFrame(data=esb_solutions).transpose(),\
                    pd.DataFrame(data=pcddr_solutions).transpose()], axis=1)